## Evaluation of multiple models

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
sns.set()

In [2]:
from sklearn.metrics import roc_auc_score, accuracy_score

def compute_metrics(y_pred, y, protection=1e-8):
    """
    Compute accuracy, AUC score, Negative Log Loss, MSE and F1 score
    """
    # print(y_pred.min(), y_pred.max(), y_pred.shape)
    y_pred = np.array([i if np.isfinite(i) else 0.5 for i in y_pred])
    acc = accuracy_score(y, y_pred >= 0.5)
    auc = roc_auc_score(y, y_pred)
    return acc, auc


class Metrics:
    """
    Keep track of metrics over time in a dictionary.
    """
    def __init__(self):
        self.metrics = {}
        self.counts = {}

    def store(self, new_metrics):
        for key in new_metrics:
            if key in self.metrics:
                self.metrics[key] += new_metrics[key]
                self.counts[key] += 1
            else:
                self.metrics[key] = new_metrics[key]
                self.counts[key] = 1

    def average(self):
        average = {k: v / self.counts[k] for k, v in self.metrics.items()}
        self.metrics, self.counts = {}, {}
        return average


In [3]:
DATASETS = ["squirrel", "ednet_kt3", "eedi", "junyi_15"]
SEED = 888
dataset = "squirrel"

DATASET_NAMES = {
    "squirrel": "Squirrel",
    "ednet_kt3": "EdNet KT3",
    "eedi": "Eedi",
    "junyi_15": "Junyi15"
}

splits = []
for i in range(5):
    path = "./../../data/" + dataset + "/preparation/split_s" + str(SEED) + "_" + str(i) + ".pkl"
    with open(path, "rb") as file_object:
        s = pickle.load(file_object)
    splits.append(s)


In [4]:
# Determine ACC and AUC of multi model evaluation

def get_eval(p, dataset, suf, splits, split_id):
    path = "partitioning/" + dataset + "_" + p + "_s" + str(split_id) + "_" + suf + ".pkl"
    train_selector = splits[split_id]["selector_train"]
    test_selector = splits[split_id]["selector_test"]
    
    with open(path, 'rb') as f:
        res_dict = pickle.load(f)

    y_pred_tr = res_dict["y_pred_train"][train_selector]
    y_truth_tr = res_dict["y_truth_train"][train_selector]
    y_pred_te = res_dict["y_pred_test"][test_selector]
    y_truth_te = res_dict["y_truth_teest"][test_selector]

    acc_train, auc_train = \
            compute_metrics(y_pred_tr, y_truth_tr)

    acc_test, auc_test = \
        compute_metrics(y_pred_te, y_truth_te)

    return acc_train, auc_train, acc_test, auc_test


In [5]:
def comb_lr_performance(ps, dataset, suf, splits, split_id):
    
    # combine predictions
    train_selector = splits[split_id]["selector_train"]
    test_selector = splits[split_id]["selector_test"]

    pred_tr, pred_te = [], []
    for p in ps:
        print(p)
        path = "partitioning/" + dataset + "_" + p + "_s" + str(split_id) + "_" + suf + ".pkl"
        with open(path, 'rb') as f:
            res_dict = pickle.load(f)

        y_pred_tr = res_dict["y_pred_train"][train_selector]
        y_truth_tr = res_dict["y_truth_train"][train_selector]
        y_pred_te = res_dict["y_pred_test"][test_selector]
        y_truth_te = res_dict["y_truth_teest"][test_selector]
        
        pred_tr.append(y_pred_tr) 
        pred_te.append(y_pred_te)

    X_train = np.array(pred_tr).T
    y_train = y_truth_tr
    X_test = np.array(pred_te).T
    y_test = y_truth_te

    lr_model = LogisticRegression(solver="liblinear",
                            max_iter=5000,
                            n_jobs=8,
                            verbose=1)
    lr_model.fit(X_train, y_train)

    pred_tr = lr_model.predict_proba(X_train)[:, 1]
    pred_te = lr_model.predict_proba(X_test)[:, 1]

    acc_train, auc_train = \
        compute_metrics(pred_tr, y_train)
    acc_test, auc_test = \
        compute_metrics(pred_te, y_test)

    return acc_train, auc_train, acc_test, auc_test

### Individual split performance

In [ ]:
### Best-LR
suf = "i_s_scA_scW_tcA_tcW"
# squirrel
partitions = ["single", "time", "i", "s", "sm", "tea", "sch", "c", "t", "at"]
# ednet
# partitions = ["single", "time", "i", "hashed_skill_id", "sm", "bundle_id", "part_id", "at"]
# eedi
# ["single", "time", "i", "hashed_skill_id", "sm", "tea", "bundle_id"]
# junyi
# #partitions = ["single", "time", "i", "s", "sm", "part_id"]


### Informed LR
# squirrel
suf = "i_icA_TW_icW_TW_lag_time_cat_n_gram_postcA_postcW_precA_precW_prev_" \
    + "resp_time_cat_rc_s_scA_TW_scW_TW_sm_t_tcA_TW_tcW_TW_user_avg_correct_vw"
# ednet
# suf = "i_icA_TW_icW_TW_lag_time_cat_n_gram_partcA_partcW_" \
#     + "prev_resp_time_cat_s_scA_TW_scW_TW_sm_tcA_TW_tcW_TW_user_avg_correct_vw"
# partitions = ["single", "time", "i", "hashed_skill_id", "sm", "bundle_id", "part_id", "at"]
# eedi
# suf = "bundle_i_icA_TW_icW_TW_n_gram_precA_precW_s_scA_TW_scW_TW_sm" \
#     + "_tcA_TW_tcW_TW_tea_user_avg_correct"
# junyi
#suf = "hour_i_icA_TW_icW_TW_lag_time_cat_n_gram_postcA_postcW_precA_" \
#    + "precW_prev_resp_time_cat_rc_s_scA_TW_scW_TW_sm_tcA_TW_tcW_TW_" \
#    + "user_avg_correct"
# partitions = ["c", "t", "at"]

print(dataset)
for p in partitions:
    print("Partition: " + p)
    print("")
    acc_vals, auc_vals = [], []
    for split_id in range(5):  #  range(5)  [0, 1, 3, 4]
        print("Split", split_id)
        acc_train, auc_train, acc_test, auc_test = \
            get_eval(p, dataset, suf, splits, split_id)
        acc_vals.append(acc_test)
        auc_vals.append(auc_test)

    acc_vals = np.array(acc_vals)
    auc_vals = np.array(auc_vals)
    print(acc_vals)
    print(auc_vals)

    acc_avg = np.round(np.mean(acc_vals), decimals=6)
    acc_std = np.round(np.std(acc_vals), decimals=6)
    auc_avg = np.round(np.mean(auc_vals), decimals=6)
    auc_std = np.round(np.std(auc_vals), decimals=6)

    out = "\\avgvar{" + str(acc_avg) + "}{" + str(acc_std) + "} &"
    out += " \\avgvar{" + str(auc_avg) + "}{" + str(auc_std) + "} \n\n"
    print(out)

    print("\n---------------------------------------\n")




### Higher LR performance

In [6]:
### Best-LR
suf = "i_s_scA_scW_tcA_tcW"
# squirrel
ps = ['s', 'sm', 'c', 'at']
# ednet
# ps = ['single', 'time', 'hashed_skill_id', 'sm', 'bundle_id', 'at'] 
# eedi
# ps = ['time', 'i', 'bundle_id']
# junyi_15
# ps = ['single', 'time', 'i', 'sm', 'part_id']

# InformedLR
# squirrel 
# suf = "i_icA_TW_icW_TW_lag_time_cat_n_gram_postcA_postcW_precA_precW_prev_"\
#     + "resp_time_cat_rc_s_scA_TW_scW_TW_sm_t_tcA_TW_tcW_TW_user_avg_correct_vw"
# ps = ["sm", "c"]
# ednet
# suf = "i_icA_TW_icW_TW_lag_time_cat_n_gram_partcA_partcW_" \
#       "prev_resp_time_cat_s_scA_TW_scW_TW_sm_tcA_TW_tcW_TW_user_avg_correct_vw"
# ps = ['time', 'sm']
# eedi
# suf = "bundle_i_icA_TW_icW_TW_n_gram_precA_precW_s_scA_TW_scW_TW_sm" + \
#     "_tcA_TW_tcW_TW_tea_user_avg_correct"
# ps = ['time', 'sm']
# junyi
# suf = "hour_i_icA_TW_icW_TW_lag_time_cat_n_gram_postcA_postcW_precA_" \
#     + "precW_prev_resp_time_cat_rc_s_scA_TW_scW_TW_sm_tcA_TW_tcW_TW_" \
#     + "user_avg_correct"
# ps = ['time', 'sm', 'part_id']


print(dataset)
print("-----------------------------")
acc_vals, auc_vals = [], []
for split_id in range(5):
    print("Split", split_id)
    acc_train, auc_train, acc_test, auc_test = \
        comb_lr_performance(ps, dataset, suf, splits, split_id)
    acc_vals.append(acc_test)
    auc_vals.append(auc_test)

acc_vals = np.array(acc_vals)
auc_vals = np.array(auc_vals)
print(acc_vals)
print(auc_vals)

acc_avg = np.round(np.mean(acc_vals), decimals=6)
acc_std = np.round(np.std(acc_vals), decimals=6)
auc_avg = np.round(np.mean(auc_vals), decimals=6)
auc_std = np.round(np.std(auc_vals), decimals=6)

print("\n---------------------------------------\n")

out = "\\avgvar{" + str(acc_avg) + "}{" + str(acc_std) + "} &"
out += " \\avgvar{" + str(auc_avg) + "}{" + str(auc_std) + "} \n\n"

print(ps)
print("\n---------------------------------------\n")
print(out)

squirrel
-----------------------------
Split 0
s
sm
c
at


/usr0/home/rschmuck/miniconda3/envs/vedu/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1357: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]Split 1
s
sm
c
at


/usr0/home/rschmuck/miniconda3/envs/vedu/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1357: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]Split 2
s
sm
c
at


...

[LibLinear]Split 3
s
sm
c
at


/usr0/home/rschmuck/miniconda3/envs/vedu/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1357: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]Split 4
s
sm
c
at


/usr0/home/rschmuck/miniconda3/envs/vedu/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1357: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear][0.76083299 0.76063921 0.76062471 0.76033632 0.76147323]
[0.79170883 0.79126132 0.79139794 0.79113294 0.79119533]

---------------------------------------

['s', 'sm', 'c', 'at']

---------------------------------------

\avgvar{0.760781}{0.000381} & \avgvar{0.791339}{0.000205} 


